In [2]:
import numpy as np
import tensorflow as tf
import scipy.io as sio 
import os 

In [3]:
def get_train_dataset():
    with tf.name_scope('traindata'):
        matx = sio.loadmat('C://Users//59723//Desktop//hello//p00//day01.mat')
        data = matx['data']
        l_data = data['left']
        r_data = data['right']
        img_l = l_data[0,0]['image'][0,0].astype(np.float32)
        gaz_l = l_data[0,0]['gaze'][0,0]
        pos_l = l_data[0,0]['pose'][0,0]
        img_r = r_data[0,0]['image'][0,0].astype(np.float32)
        gaz_r = r_data[0,0]['gaze'][0,0]
        pos_r = r_data[0,0]['pose'][0,0]
        img_l = np.reshape(img_l, [-1, 36*60])
        img_r = np.reshape(img_r, [-1, 36*60])
        gazelr = np.concatenate((gaz_l, gaz_r), axis=1).astype(np.float32)
        poselr = np.concatenate((pos_l, pos_r), axis=1).astype(np.float32)
        gazeset = tf.data.Dataset.from_tensor_slices(
            {
                'img_l':img_l,
                'img_r':img_r,
                'gaze':gazelr,
                'pose':poselr
            }
        )
        gazeset = gazeset.shuffle(1000)
        gazeset = gazeset.repeat()
        gazeset = gazeset.batch(16)
        iterator = gazeset.make_one_shot_iterator()
        trainset = iterator.get_next()
    return trainset

In [4]:
def get_test_dataset():
    with tf.name_scope('testdata'):
        matx = sio.loadmat('C://Users//59723//Desktop//hello//p00//day02.mat')
        data = matx['data']
        l_data = data['left']
        r_data = data['right']
        img_l = l_data[0,0]['image'][0,0].astype(np.float32)
        gaz_l = l_data[0,0]['gaze'][0,0]
        pos_l = l_data[0,0]['pose'][0,0]
        img_r = r_data[0,0]['image'][0,0].astype(np.float32)
        gaz_r = r_data[0,0]['gaze'][0,0]
        pos_r = r_data[0,0]['pose'][0,0]
        img_l = np.reshape(img_l, [-1, 36*60])
        img_r = np.reshape(img_r, [-1, 36*60])
        gazelr = np.concatenate((gaz_l, gaz_r), axis=1).astype(np.float32)
        poselr = np.concatenate((pos_l, pos_r), axis=1).astype(np.float32)
        gazeset = tf.data.Dataset.from_tensor_slices(
            {
                'img_l':img_l,
                'img_r':img_r,
                'gaze':gazelr,
                'pose':poselr
            }
        )
        gazeset = gazeset.shuffle(1000)
        gazeset = gazeset.repeat()
        gazeset = gazeset.batch(16)
        iterator = gazeset.make_one_shot_iterator()
        testset = iterator.get_next()
    return testset

Base-Net

In [5]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1],
                     padding='VALID', name='conv')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x, name='relu')
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME', name='pool')

In [6]:
def BaseNet(inputs):
    with tf.name_scope('input_layer'):
        input_layer = tf.reshape(inputs, [-1, 36, 60, 1])
    with tf.name_scope('conv1'):
        w1 = tf.Variable(tf.random_normal([3, 3, 1, 64]), name='w1')
        b1 = tf.Variable(tf.random_normal([64]), name='b1')
        conv1 = conv2d(input_layer, w1, b1, strides=1)
    with tf.name_scope('conv2'):
        w2 = tf.Variable(tf.random_normal([3, 3, 64, 64]), name='w2')
        b2 = tf.Variable(tf.random_normal([64]), name='b2')
        conv2 = conv2d(conv1, w2, b2, strides=1)
    with tf.name_scope('maxpool1'):
        pool1 = maxpool2d(conv2, k=2)
    with tf.name_scope('conv3'):
        w3 = tf.Variable(tf.random_normal([3, 3, 64, 128]), name='w3')
        b3 = tf.Variable(tf.random_normal([128]), name='b3')
        conv3 = conv2d(pool1, w3, b3, strides=1)
    with tf.name_scope('conv4'):
        w4 = tf.Variable(tf.random_normal([3, 3, 128, 128]), name='w4')
        b4 = tf.Variable(tf.random_normal([128]), name='b4')
        conv4 = conv2d(conv3, w4, b4, strides=1)
    with tf.name_scope('maxpool2'):
        pool2 = maxpool2d(conv4, k=2)
    with tf.name_scope('conv5'):
        w5 = tf.Variable(tf.random_normal([3, 3, 128, 256]), name='w5')
        b5 = tf.Variable(tf.random_normal([256]), name='b5')
        conv5 = conv2d(pool2, w5, b5, strides=1)
    with tf.name_scope('conv6'):
        w6 = tf.Variable(tf.random_normal([3, 3, 256, 256]), name='w6')
        b6 = tf.Variable(tf.random_normal([256]), name='b6')
        conv6 = conv2d(conv5, w6, b6, strides=1)
    with tf.name_scope('maxpool3'):
        pool3 = maxpool2d(conv6, k=2)
    return pool3

AR-Net

In [7]:
def ARNet(input_l, input_r, input_p):
    with tf.name_scope('stream1'):
        with tf.name_scope('basenet1'):
            basenet1 = BaseNet(input_l)
        with tf.name_scope('fc11'):
            wd11 = tf.Variable(tf.random_normal([1*4*256, 1000]), name='wd11')
            bd11 = tf.Variable(tf.random_normal([1000]), name='bd11')
            fc11 = tf.reshape(basenet1, [-1, wd11.get_shape().as_list()[0]])
            fc11 = tf.add(tf.matmul(fc11, wd11), bd11)
            fc11 = tf.nn.relu(fc11)
        with tf.name_scope('fc12'):
            wd12 = tf.Variable(tf.random_normal([1000, 500]), name='wd12')
            bd12 = tf.Variable(tf.random_normal([500]), name='bd12')
            fc12 = tf.reshape(fc11, [-1, wd12.get_shape().as_list()[0]])
            fc12 = tf.add(tf.matmul(fc12, wd12), bd12)
            fc12 = tf.nn.relu(fc12)
        st1 = fc12
    with tf.name_scope('stream2'):
        with tf.name_scope('basenet2'):
            basenet2 = BaseNet(input_r)
        with tf.name_scope('fc21'):
            wd21 = tf.Variable(tf.random_normal([1*4*256, 1000]), name='wd21')
            bd21 = tf.Variable(tf.random_normal([1000]), name='bd21')
            fc21 = tf.reshape(basenet2, [-1, wd21.get_shape().as_list()[0]])
            fc21 = tf.add(tf.matmul(fc21, wd21), bd21)
            fc21 = tf.nn.relu(fc21)
        with tf.name_scope('fc22'):
            wd22 = tf.Variable(tf.random_normal([1000, 500]), name='wd22')
            bd22 = tf.Variable(tf.random_normal([500]), name='bd22')
            fc22 = tf.reshape(fc21, [-1, wd22.get_shape().as_list()[0]])
            fc22 = tf.add(tf.matmul(fc22, wd22), bd22)
            fc22 = tf.nn.relu(fc22)
        st2 = fc22
    with tf.name_scope('stream3_4'):
        with tf.name_scope('basenet3'):
            basenet3 = BaseNet(input_l)
        with tf.name_scope('fc31'):
            wd31 = tf.Variable(tf.random_normal([1*4*256, 500]), name='wd31')
            bd31 = tf.Variable(tf.random_normal([500]), name='bd31')
            fc31 = tf.reshape(basenet3, [-1, wd31.get_shape().as_list()[0]])
            fc31 = tf.add(tf.matmul(fc31, wd31), bd31)
            fc31 = tf.nn.relu(fc31)
        with tf.name_scope('basenet4'):
            basenet4 = BaseNet(input_r)
        with tf.name_scope('fc41'):
            wd41 = tf.Variable(tf.random_normal([1*4*256, 500]), name='wd41')
            bd41 = tf.Variable(tf.random_normal([500]), name='bd41')
            fc41 = tf.reshape(basenet4, [-1, wd41.get_shape().as_list()[0]])
            fc41 = tf.add(tf.matmul(fc41, wd41), bd41)
            fc41 = tf.nn.relu(fc41)
        with tf.name_scope('merge'):
            fc34 = tf.concat([fc31, fc41], axis=1)
            wd34 = tf.Variable(tf.random_normal([1000, 500]), name='wd34')
            bd34 = tf.Variable(tf.random_normal([500]), name='bd34')
            fc34 = tf.reshape(fc34, [-1, wd34.get_shape().as_list()[0]])
            fc34 = tf.add(tf.matmul(fc34, wd34), bd34)
            fc34 = tf.nn.relu(fc34)
        st34 = fc34
    with tf.name_scope('3D_Gaze'):
        st = tf.concat([st1, st2, st34, input_p], axis=1)
        wd = tf.Variable(tf.random_normal([1506, 6]), name='wd')
        bd = tf.Variable(tf.random_normal([6]), name='bd')
        gaze = tf.add(tf.matmul(st, wd), bd)
    return gaze

In [8]:
def tenlen(inputs):
    with tf.name_scope('length'):
        shape = inputs.get_shape().as_list()
        lenth = tf.sqrt(tf.reduce_sum(tf.square(inputs),axis=1))
        lenth = tf.reshape(lenth, [-1, 1])
        lenth = tf.tile(lenth, [1, shape[1]])
    return lenth

E-Net

In [9]:
def ENet(input_l, input_r):
    with tf.name_scope('stream1'):
        with tf.name_scope('basenet1'):
            basenet1 = BaseNet(input_l)
        with tf.name_scope('fc11'):
            wd11 = tf.Variable(tf.random_normal([1*4*256, 1000]), name='wd11')
            bd11 = tf.Variable(tf.random_normal([1000]), name='bd11')
            fc11 = tf.reshape(basenet1, [-1, wd11.get_shape().as_list()[0]])
            fc11 = tf.add(tf.matmul(fc11, wd11), bd11)
            fc11 = tf.nn.relu(fc11)
        with tf.name_scope('fc12'):
            wd12 = tf.Variable(tf.random_normal([1000, 500]), name='wd12')
            bd12 = tf.Variable(tf.random_normal([500]), name='bd12')
            fc12 = tf.reshape(fc11, [-1, wd12.get_shape().as_list()[0]])
            fc12 = tf.add(tf.matmul(fc12, wd12), bd12)
            fc12 = tf.nn.relu(fc12)
        st1 = fc12
    with tf.name_scope('stream2'):
        with tf.name_scope('basenet2'):
            basenet2 = BaseNet(input_r)
        with tf.name_scope('fc21'):
            wd21 = tf.Variable(tf.random_normal([1*4*256, 1000]), name='wd21')
            bd21 = tf.Variable(tf.random_normal([1000]), name='bd21')
            fc21 = tf.reshape(basenet2, [-1, wd21.get_shape().as_list()[0]])
            fc21 = tf.add(tf.matmul(fc21, wd21), bd21)
            fc21 = tf.nn.relu(fc21)
        with tf.name_scope('fc22'):
            wd22 = tf.Variable(tf.random_normal([1000, 500]), name='wd22')
            bd22 = tf.Variable(tf.random_normal([500]), name='bd22')
            fc22 = tf.reshape(fc21, [-1, wd22.get_shape().as_list()[0]])
            fc22 = tf.add(tf.matmul(fc22, wd22), bd22)
            fc22 = tf.nn.relu(fc22)
        st2 = fc22
    with tf.name_scope('softmax'):
        st = tf.concat([st1, st2], axis=1)
        wd = tf.Variable(tf.random_normal([1000, 2]), name='wd')
        bd = tf.Variable(tf.random_normal([2]), name='bd')
        fc = tf.add(tf.matmul(st, wd), bd)
        sf = tf.nn.softmax(fc, name='fun_softmax')
    return sf

In [19]:
rate = 0.005
training_iters = 500
batch_size = 16
beta = 0.1
display_step = 10
k = tf.Graph()
with k.as_default():
    input_l = tf.placeholder(tf.float32, [None, 36*60], name='L_img')
    input_r = tf.placeholder(tf.float32, [None, 36*60], name='R_img')
    input_g = tf.placeholder(tf.float32, [None, 6], name='gaze')
    input_p = tf.placeholder(tf.float32, [None, 6], name='pose')
    trainset = get_train_dataset()
    testset = get_test_dataset()
    with tf.name_scope('AR_Net'):
        gaze = ARNet(input_l, input_r, input_p)
        f_l = gaze[:,0:3]
        f_l = tf.math.divide(f_l, tenlen(f_l))
        f_r = gaze[:,3:6]
        f_r = tf.math.divide(f_r, tenlen(f_r))
        g_l = input_g[:,0:3]
        g_l = tf.math.divide(g_l, tenlen(g_l))
        g_r = input_g[:,3:6]
        g_r = tf.math.divide(g_r, tenlen(g_r))
        with tf.name_scope('AR_loss'):
            e_l = tf.math.acos(tf.reduce_sum(f_l*g_l, axis=1))/np.pi*180
            e_r = tf.math.acos(tf.reduce_sum(f_r*g_r, axis=1))/np.pi*180
            Lar = tf.math.divide(2*(e_l*e_r),(e_l+e_r))
    with tf.name_scope('E_Net'):
        prob = ENet(input_l, input_r)
        p_l = prob[:,0:1]
        p_l = tf.reshape(p_l, [-1])
        p_r = prob[:,1:2]
        p_r = tf.reshape(p_r, [-1])
        with tf.name_scope('E_loss'):
            e = tf.math.acos(tf.reduce_sum(f_l*f_r, axis=1))
            nu = tf.greater(e_r, e_l)
            nu = tf.cast(nu, dtype=tf.float32)
            Le = -1*(nu*e*tf.math.log(p_l)+(1-nu)*e*tf.math.log(p_r))
    with tf.name_scope('ARE_loss'):``
        w = (1+(2*nu-1)*p_l+(1-2*nu)*p_r)/2
        Lare = w*Lar+(1-w)*beta*(e_l+e_r)/2
        Lare = tf.reduce_mean(Lare)
    with tf.name_scope('angular_loss'):
        ang_loss = tf.reduce_mean((e_l+e_r)/2)
    with tf.name_scope('op_Lare'):
        op_Lare = tf.train.AdamOptimizer(learning_rate=rate).minimize(Lare)
    with tf.name_scope('op_Le'):
        op_Le = tf.train.AdamOptimizer(learning_rate=rate).minimize(Le)
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        writer = tf.summary.FileWriter('log/', sess.graph)
        loss_summary = tf.summary.scalar('loss', Lare)
        sess.run(init)
        step = 1
        while step<=training_iters:
            batch = sess.run(trainset)
            #print(batch['gaze'][0])
            sess.run(op_Lare, feed_dict={
                input_l:batch['img_l'],
                input_r:batch['img_r'],
                input_g:batch['gaze'],
                input_p:batch['pose']
            })
            '''
            sess.run(op_Le, feed_dict={
                input_l:batch['img_l'],
                input_r:batch['img_r'],
                input_g:batch['gaze'],
                input_p:batch['pose']
            })
            '''
            [e_ll, e_rr] = sess.run([e_l, e_r],
                feed_dict={
                    input_l:batch['img_l'],
                    input_r:batch['img_r'],
                    input_g:batch['gaze'],
                    input_p:batch['pose']
                })
            [lare, le, acc] = sess.run([Lare, Le, ang_loss],
                feed_dict={
                    input_l:batch['img_l'],
                    input_r:batch['img_r'],
                    input_g:batch['gaze'],
                    input_p:batch['pose']
                })
            [f_ll, g_ll] = sess.run([f_l, g_l],
                feed_dict={
                    input_l:batch['img_l'],
                    input_r:batch['img_r'],
                    input_g:batch['gaze'],
                    input_p:batch['pose']
                })
            #if step%display_step==0:
            print('iter '+str(step)+' lare='+'{:.6f}'.format(lare))
            #print('    le='+'{:.6f}'.format(le))
            print('    ang_loss='+'{:.6f}'.format(acc))
            print('    e_l='+'{:.6f}'.format(e_ll[0])+' e_r='+'{:.6f}'.format(e_rr[0]))
            print(g_ll[0])
            print(f_ll[0])
            #print(''+'{:.6f}'.format(g_ll[0][0]))
            #print('    func='+'{:.6f}'.format(f_ll[0][0]))
            step += 1
        batch = sess.run(testset)
        print('train over')
        [finalloss, finalacc] = sess.run([Lare, ang_loss], feed_dict={
                input_l:batch['img_l'],
                input_r:batch['img_r'],
                input_g:batch['gaze'],
                input_p:batch['pose']
        })
        print('finalloss='+'{:.6f}'.format(finalloss))
        print('finalangloss='+'{:.6f}'.format(finalacc))

iter 1 lare=7.345617
    ang_loss=10.785369
    e_l=12.429922 e_r=7.161823
[ 1.8637333e-04  2.0173751e-01 -9.7943968e-01]
[-0.18040155  0.08229099 -0.98014456]
iter 2 lare=4.452359
    ang_loss=7.703822
    e_l=6.801436 e_r=6.227118
[-0.01110249  0.12203338 -0.99246395]
[-0.12676908  0.14587764 -0.9811471 ]
iter 3 lare=2.888388
    ang_loss=8.298311
    e_l=13.671021 e_r=13.383117
[ 0.07195368  0.25098142 -0.96531403]
[-0.09815346  0.08652573 -0.9914027 ]
iter 4 lare=6.911455
    ang_loss=10.215003
    e_l=12.848655 e_r=12.351715
[ 0.15739444  0.11315145 -0.981032  ]
[-0.05989055  0.06183949 -0.99628764]
iter 5 lare=2.218468
    ang_loss=8.559094
    e_l=6.869212 e_r=9.170676
[-0.01035973  0.22030288 -0.97537655]
[-0.00702779  0.10210732 -0.9947486 ]
iter 6 lare=3.357405
    ang_loss=8.297204
    e_l=4.020975 e_r=8.471064
[ 0.14423662  0.15904003 -0.97667915]
[ 0.10653075  0.10119604 -0.9891463 ]
iter 7 lare=2.114450
    ang_loss=10.931007
    e_l=23.634459 e_r=14.136979
[-0.22010969  

iter 56 lare=1.467034
    ang_loss=8.702652
    e_l=1.658002 e_r=3.428626
[-0.136148    0.25400487 -0.95757264]
[-0.13715704  0.22591154 -0.96444386]
iter 57 lare=0.876468
    ang_loss=8.764680
    e_l=17.147820 e_r=14.673045
[ 0.14179234  0.08191883 -0.9865011 ]
[-0.08561659  0.27269244 -0.95828414]
iter 58 lare=1.778693
    ang_loss=12.443648
    e_l=14.560532 e_r=11.558648
[-0.03216054  0.06857952 -0.99712723]
[-0.12984413  0.2968036  -0.9460699 ]
iter 59 lare=1.217556
    ang_loss=12.175558
    e_l=13.431823 e_r=10.575943
[ 0.05582704  0.14019574 -0.98854876]
[-0.11951814  0.28998947 -0.94953746]
iter 60 lare=1.842241
    ang_loss=12.119143
    e_l=17.143127 e_r=15.599497
[ 0.15671179  0.21674208 -0.96356857]
[-0.13241972  0.28769162 -0.94852436]
iter 61 lare=1.533708
    ang_loss=11.649793
    e_l=12.571198 e_r=10.613575
[-0.12529774  0.04399159 -0.99114347]
[-0.14635165  0.25888744 -0.9547557 ]
iter 62 lare=1.156052
    ang_loss=11.560518
    e_l=9.245924 e_r=9.067286
[-0.2541113

iter 111 lare=0.793866
    ang_loss=7.938656
    e_l=3.728776 e_r=3.605869
[-0.16641884  0.2363705  -0.95730555]
[-0.10343197  0.224856   -0.96888685]
iter 112 lare=1.756334
    ang_loss=7.083865
    e_l=10.581449 e_r=7.530356
[-0.2391965   0.06382737 -0.9688711 ]
[-0.1402192   0.21939978 -0.9655063 ]
iter 113 lare=1.675237
    ang_loss=9.186295
    e_l=14.278016 e_r=11.947201
[ 0.18178912  0.23543069 -0.9547383 ]
[-0.06631982  0.2422874  -0.96793514]
iter 114 lare=1.086508
    ang_loss=7.891042
    e_l=9.571565 e_r=6.013532
[-0.21050951  0.1140641  -0.97091466]
[-0.1167131   0.25169238 -0.96074396]
iter 115 lare=1.917615
    ang_loss=9.156017
    e_l=11.767216 e_r=8.102901
[-0.23211578  0.09251077 -0.96827894]
[-0.09973934  0.24898343 -0.9633583 ]
iter 116 lare=1.868302
    ang_loss=8.977322
    e_l=13.001837 e_r=10.221543
[-0.07479084  0.01161125 -0.9971317 ]
[-0.04112214  0.23404434 -0.9713559 ]
iter 117 lare=0.949902
    ang_loss=9.499023
    e_l=10.039206 e_r=8.323078
[ 0.13246387

iter 166 lare=1.958708
    ang_loss=6.887131
    e_l=5.680651 e_r=6.000838
[ 0.01828005  0.27346715 -0.9617077 ]
[ 0.0210939   0.17694022 -0.98399556]
iter 167 lare=1.016818
    ang_loss=6.630627
    e_l=10.731941 e_r=8.184317
[ 0.13678469  0.00139307 -0.9905999 ]
[-0.00101932  0.1278466  -0.99179345]
iter 168 lare=0.686326
    ang_loss=4.652367
    e_l=6.475173 e_r=4.760492
[-0.23064832  0.09532799 -0.9683564 ]
[-0.13952512  0.16151756 -0.9769569 ]
iter 169 lare=1.266935
    ang_loss=5.802367
    e_l=3.169038 e_r=3.355380
[-0.21084404  0.20864357 -0.9549936 ]
[-0.17160629  0.1726417  -0.9699207 ]
iter 170 lare=1.636910
    ang_loss=5.634400
    e_l=3.149270 e_r=0.973777
[-0.12503314  0.10851333 -0.98620063]
[-0.09585714  0.15499774 -0.9832534 ]
iter 171 lare=2.041227
    ang_loss=5.693233
    e_l=9.295635 e_r=5.328331
[ 0.15988015  0.24750882 -0.9556034 ]
[ 0.01471905  0.18101396 -0.9833704 ]
iter 172 lare=1.412807
    ang_loss=6.280974
    e_l=9.585193 e_r=8.184269
[-0.14574727  0.01

iter 221 lare=2.171681
    ang_loss=8.511777
    e_l=14.678856 e_r=11.269754
[-0.23494224 -0.05078036 -0.970682  ]
[-0.28775075  0.19694912 -0.93723553]
iter 222 lare=0.717643
    ang_loss=7.176428
    e_l=12.439296 e_r=7.885506
[-0.1457527  -0.00461933 -0.9893103 ]
[-0.2711655   0.16715983 -0.9479066 ]
iter 223 lare=1.416815
    ang_loss=9.186803
    e_l=14.717042 e_r=11.111169
[ 0.12072912  0.16074306 -0.97958475]
[-0.13302945  0.12595691 -0.9830758 ]
iter 224 lare=1.470958
    ang_loss=7.123386
    e_l=13.423204 e_r=10.116140
[ 0.05191118  0.16126241 -0.9855454 ]
[-0.18105656  0.17004359 -0.9686608 ]
iter 225 lare=1.145902
    ang_loss=7.975774
    e_l=11.332891 e_r=5.855246
[ 0.06441449  0.16104493 -0.98484284]
[-0.1320624   0.14157979 -0.9810784 ]
iter 226 lare=1.232498
    ang_loss=6.433372
    e_l=12.010533 e_r=6.003842
[ 0.12941174  0.00875938 -0.9915523 ]
[-0.07051908  0.07037909 -0.99502456]
iter 227 lare=1.827538
    ang_loss=5.703510
    e_l=9.361452 e_r=4.555350
[ 0.162814

iter 275 lare=1.154057
    ang_loss=5.526562
    e_l=6.175427 e_r=4.216354
[ 0.06203414  0.10044333 -0.99300706]
[-0.04444153  0.11681698 -0.9921587 ]
iter 276 lare=1.821018
    ang_loss=5.502888
    e_l=5.165926 e_r=4.733503
[-0.15251681  0.04225494 -0.9873972 ]
[-0.0970755   0.11330483 -0.9888065 ]
iter 277 lare=1.797343
    ang_loss=6.581379
    e_l=9.869322 e_r=11.077342
[-0.23860274  0.23639861 -0.9419047 ]
[-0.15728804  0.09057542 -0.98339033]
iter 278 lare=1.976363
    ang_loss=5.898429
    e_l=8.557494 e_r=3.358996
[ 5.7653338e-04  1.2292657e-02 -9.9992436e-01]
[-0.14253609  0.05293291 -0.9883733 ]
iter 279 lare=1.488332
    ang_loss=6.818387
    e_l=3.827866 e_r=1.826492
[-0.09177844  0.08286486 -0.99232566]
[-0.15213789  0.10964793 -0.9822584 ]
iter 280 lare=0.704667
    ang_loss=7.046669
    e_l=11.400902 e_r=6.738076
[ 0.06045415  0.23718694 -0.96958125]
[-0.11566941  0.14620067 -0.9824693 ]
iter 281 lare=1.153880
    ang_loss=6.280976
    e_l=15.898973 e_r=5.324173
[ 0.148

iter 330 lare=0.853242
    ang_loss=7.065866
    e_l=7.519195 e_r=5.777739
[ 0.07620016  0.20536625 -0.97571427]
[ 0.01347271  0.3172122  -0.94825894]
iter 331 lare=0.853821
    ang_loss=7.537776
    e_l=11.223105 e_r=3.856864
[-0.19671507  0.13893047 -0.9705677 ]
[-0.08103145  0.29550466 -0.9518986 ]
iter 332 lare=1.175723
    ang_loss=6.786675
    e_l=11.315482 e_r=6.077799
[ 0.18421325  0.08394878 -0.9792948 ]
[ 0.10342816  0.2626991  -0.95931846]
iter 333 lare=1.260169
    ang_loss=5.742251
    e_l=9.984576 e_r=1.852768
[ 0.05355139  0.07583197 -0.99568164]
[ 0.08236187  0.24489675 -0.9660446 ]
iter 334 lare=0.991953
    ang_loss=6.267968
    e_l=8.438057 e_r=4.830388
[-0.17517667  0.06338421 -0.98249465]
[-0.1326735   0.2036971  -0.97000283]
iter 335 lare=1.645205
    ang_loss=6.079825
    e_l=3.109546 e_r=1.882632
[-0.16519727  0.14199011 -0.97598606]
[-0.12404208  0.17735696 -0.9762981 ]
iter 336 lare=0.811219
    ang_loss=5.160236
    e_l=2.863753 e_r=0.834391
[-0.27628288  0.1

iter 385 lare=1.684233
    ang_loss=4.037228
    e_l=3.131379 e_r=2.595330
[ 0.10543746  0.16640323 -0.9804045 ]
[ 0.15855673  0.15469192 -0.9751565 ]
iter 386 lare=1.448293
    ang_loss=4.062025
    e_l=6.288058 e_r=3.886804
[ 0.07983303  0.1598351  -0.9839103 ]
[ 0.18821175  0.14922464 -0.97072566]
iter 387 lare=1.220379
    ang_loss=4.655259
    e_l=5.289168 e_r=3.641854
[ 0.03271635  0.2675235  -0.9629958 ]
[-0.01168317  0.18887284 -0.98193204]
iter 388 lare=1.170246
    ang_loss=4.419191
    e_l=3.016555 e_r=2.468072
[ 0.15082347  0.19893764 -0.9683368 ]
[ 0.10189545  0.18207681 -0.9779905 ]
iter 389 lare=0.515804
    ang_loss=4.048747
    e_l=3.123053 e_r=2.279244
[ 0.05355139  0.07583197 -0.99568164]
[ 0.01772561  0.11681876 -0.9929951 ]
iter 390 lare=1.159752
    ang_loss=5.510861
    e_l=13.413123 e_r=12.332798
[ 0.15671179  0.21674208 -0.96356857]
[-0.06233498  0.13951793 -0.98825556]
iter 391 lare=1.362279
    ang_loss=5.409636
    e_l=2.753992 e_r=1.664481
[-0.26626045  0.0

iter 440 lare=2.582705
    ang_loss=5.552190
    e_l=1.573709 e_r=2.893809
[ 0.11301076  0.22131011 -0.96863335]
[ 0.11219462  0.19447446 -0.9744703 ]
iter 441 lare=1.946528
    ang_loss=3.448708
    e_l=1.709033 e_r=0.976787
[ 0.05191118  0.16126241 -0.9855454 ]
[ 0.03897724  0.18782689 -0.98142856]
iter 442 lare=1.655464
    ang_loss=3.985500
    e_l=1.913564 e_r=1.482265
[-0.03443738  0.24772088 -0.9682193 ]
[-0.04145618  0.21589673 -0.9755358 ]
iter 443 lare=0.561769
    ang_loss=4.459485
    e_l=6.754734 e_r=4.472557
[ 0.10387819  0.03478807 -0.9939815 ]
[ 0.05804998  0.14316966 -0.9879943 ]
iter 444 lare=0.906053
    ang_loss=5.630291
    e_l=1.583626 e_r=2.271244
[-0.19772467  0.20005839 -0.95962584]
[-0.21784393  0.217189   -0.9515109 ]
iter 445 lare=0.513116
    ang_loss=5.131161
    e_l=5.690096 e_r=2.324142
[-0.15889694  0.17764461 -0.97118187]
[-0.25599155  0.17244093 -0.95117426]
iter 446 lare=1.079945
    ang_loss=7.122087
    e_l=5.978354 e_r=3.219904
[-0.05105621  0.121

iter 495 lare=1.260057
    ang_loss=4.217935
    e_l=9.406422 e_r=7.372939
[ 0.14678337  0.10806771 -0.98324776]
[-0.01292525  0.14478186 -0.98937917]
iter 496 lare=0.830478
    ang_loss=3.582618
    e_l=6.997622 e_r=6.880881
[-0.14428644  0.03318242 -0.9889795 ]
[-0.1826182  0.1478135 -0.9720091]
iter 497 lare=1.222334
    ang_loss=4.574414
    e_l=2.306814 e_r=0.260196
[-0.01035973  0.22030288 -0.97537655]
[-0.02799336  0.18479434 -0.9823784 ]
iter 498 lare=1.217557
    ang_loss=3.666104
    e_l=2.624877 e_r=1.875028
[ 0.08747622  0.17937706 -0.9798836 ]
[ 0.12062016  0.14780171 -0.9816341 ]
iter 499 lare=1.077786
    ang_loss=3.924717
    e_l=4.610869 e_r=2.203981
[-0.02479122  0.21544708 -0.9762008 ]
[-0.00767662  0.13812552 -0.99038494]
iter 500 lare=0.840436
    ang_loss=5.071321
    e_l=5.521693 e_r=3.590360
[-0.08957852  0.15392095 -0.98401433]
[-0.00104145  0.11705891 -0.9931245 ]
train over
finalloss=1.401728
finalangloss=3.220872
